## Day 3: Enhanced Functions

### Arrow Functions

* Arrow functions provide a shorter syntax for writing functions and do not bind their own `this`.

#### Basic Syntax

* Arrow functions (`=>`) provide a shorter syntax for writing functions and they capture the this value of their surrounding scope.

    ```js
    // Traditional function
    function add(a, b) {
    return a + b;
    }

    // Arrow function
    const addArrow = (a, b) => a + b;

    console.log(add(5, 3));      // 8
    console.log(addArrow(5, 3)); // 8
    ```

#### Syntax Variations

```js
// No parameters
const sayHello = () => "Hello!";

// Single parameter (parentheses optional)
const double = num => num * 2;
// const double = (num) => num * 2; // Also valid

// Multiple parameters (parentheses required)
const multiply = (a, b) => a * b;

// Multiple statements (curly braces and explicit return required)
const calculateTotal = (price, tax) => {
const taxAmount = price * tax;
return price + taxAmount;
};

console.log(sayHello());          // "Hello!"
console.log(double(5));           // 10
console.log(multiply(4, 6));      // 24
console.log(calculateTotal(100, 0.08)); // 108
```

#### `this` Binding

* Arrow functions don't have their own `this` context. They inherit `this` from the surrounding code:
*  This is perfect for `callbacks`

    ```js
    <button id="btn">Click me</button>
    <script>
    const btn = document.getElementById('btn');
    btn.addEventListener('click', () => {
        // `this` here is the same `this` as outer scope (e.g. window),
        // so avoid arrow functions if you need the element itself
        console.log(this); 
    });
    </script>
    ```
* for **array methods** or **timers**, arrow functions shine:

    ```js
    // Problem with traditional functions
    const counter = {
    count: 0,
    incrementLater: function() {
        setTimeout(function() {
        this.count++; // 'this' refers to window/global object, not counter
        console.log(this.count);
        }, 1000);
    }
    };
    counter.incrementLater(); // NaN or error

    // Solution with arrow function
    const betterCounter = {
    count: 0,
    incrementLater: function() {
        setTimeout(() => {
        this.count++; // 'this' refers to betterCounter
        console.log(this.count);
        }, 1000);
    }
    };
    betterCounter.incrementLater(); // 1
    ```

* **When Not to Use**
  
  1. Object methods needing this

    ```js
    // 1. Object Method Needing `this`
    // Using an arrow function for a method means `this` is inherited
    const counter = {
    count: 0,
    // Arrow function: `this` is not `counter`!
    increment: () => {
        this.count++;
        console.log(this.count);
    },
    // Correct: use a regular function to bind `this` to `counter`
    incrementCorrect() {
        this.count++;
        console.log(this.count);
    }
    };

    counter.increment();         // NaN or error, `this.count` is undefined
    counter.incrementCorrect();  // 1
    ```

  2. Constructor functions (cannot use new with arrows)

    ```js
    // 2. Constructor Functions (Arrow Cannot Be Used with `new`)
    const Person = (name) => {
    this.name = name;
    };
    // Attempt to use `new` with an arrow will throw:
    try {
    const p = new Person('Alice');
    } catch (e) {
    console.error(e); // TypeError: Person is not a constructor
    }

    // Correct: use a regular function or class
    function PersonFunc(name) {
    this.name = name;
    }
    const p2 = new PersonFunc('Bob');
    console.log(p2.name); // 'Bob'
    ```

  3. Event handlers when you rely on this being the DOM element

    ```js
    // 3. Event Handlers Relying on `this` Being the DOM Element
    // Using an arrow function here, `this` is inherited (e.g., window), not the element
    const button = document.createElement('button');
    button.textContent = 'Click me';
    document.body.appendChild(button);

    button.addEventListener('click', () => {
    console.log(this);            // window (or undefined in strict mode)
    console.log(this.textContent); // undefined
    });

    // Correct: use a regular function expression
    button.addEventListener('click', function() {
    console.log(this);            // the <button> element
    console.log(this.textContent); // 'Click me'
    });
    ```

## Default Parameters

* Default parameters allow you to specify default values for function parameters:

  ```js
  // Before ES6
  function greet(name) {
    name = name || 'Guest';
    return `Hello, ${name}!`;
  }

  // With ES6 default parameters
  function greetES6(name = 'Guest') {
    return `Hello, ${name}!`;
  }

  console.log(greetES6());       // "Hello, Guest!"
  console.log(greetES6('Alice')); // "Hello, Alice!"

  // Default parameters can be expressions
  function calculateFee(amount, percentage = 10, minimum = amount * 0.05) {
    const fee = amount * (percentage / 100);
    return fee < minimum ? minimum : fee;
  }

  console.log(calculateFee(100));       // 10
  console.log(calculateFee(100, 5));    // 5
  console.log(calculateFee(100, 2, 3)); // 3
  ```

* If you pass `undefined`, the default **still** applies.

* Defaults should go **after** required parameters for clarity.

## Rest and Spread Operators

* ES6 introduced the three-dot syntax (`…`) for two related but distinct features:
  
  * **Rest parameters** — collect function arguments into an array  
  
  * **Spread operator** — expand an array (or iterable) into individual elements  

### Rest Operator in Functions

* **What it is:**: A more flexible, array-based replacement for the old `arguments` object.

    ```js
    // ES5: the `arguments` object (not a real array)
    function showArgsES5() {
    console.log(arguments);  // => [object Arguments] { '0': 'Alice', '1': 'Bob' }
    }

    // ES6: rest parameters, always a true Array
    function showArgsES6(...args) {
    console.log(args);       // => ['Alice', 'Bob']
    console.log(Array.isArray(args)); // => true
    }
    ```

* **Syntax**: rest must appear **last** in the parameter list.
  
* **Type**: a real `Array`—you can use `.map()`, `.filter()`, `.reduce()`, etc.

    ```js
    // Combining regular parameters with rest parameter
    function displayInfo(name, age, ...hobbies) {
    console.log(`Name: ${name}`);
    console.log(`Age: ${age}`);
    console.log(`Hobbies: ${hobbies.join(', ')}`);
    }

    displayInfo('Alice', 30, 'reading', 'hiking', 'painting');
    // Name: Alice
    // Age: 30
    // Hobbies: reading, hiking, painting
    ```

* Quiz
  
  * Write a function `sum(...numbers)` that returns the sum of any number of arguments.

    ```js
    // Rest parameter collects all remaining arguments into an array
    function sum(...numbers) {
    return numbers.reduce((total, num) => total + num, 0);
    }

    console.log(sum(1, 2, 3, 4, 5)); // 15
    ```


### Spread Operator

* **What it is**: Expands an `array` (or any iterable) into individual elements—often used in function calls.

* **Syntax**: use `...` before an `array` (or iterable) in a function call.

    ```js
    function introduce(a, b, c) {
    console.log(`Meet ${a}, ${b}, and ${c}.`);
    }

    const heroes = ['Iron Man', 'Captain America', 'Black Widow'];
    introduce(...heroes);
    // → “Meet Iron Man, Captain America, and Black Widow.”
    ```

* **Merging Arrays**
  
    ```js
    // Spread operator with arrays
    const arr1 = [1, 2, 3];
    const arr2 = [4, 5, 6];
    const combined = [...arr1, ...arr2];
    console.log(combined); // [1, 2, 3, 4, 5, 6]

    // Spread operator with objects (ES2018)
    const person = { name: 'Alice', age: 30 };
    const details = { occupation: 'Developer', location: 'New York' };
    const completePerson = { ...person, ...details };
    console.log(completePerson); 
    // { name: 'Alice', age: 30, occupation: 'Developer', location: 'New York' }
    ```
* **Cloning Arrays**
  
    ```js
    // Clone an array
    const original = [10, 20, 30];
    const copy = [...original];
    console.log(copy); // [10, 20, 30]
    ```

* **Update Property**

    ```js
    // Override properties
    const updatedPerson = { ...person, age: 31 };
    console.log(updatedPerson); // { name: 'Alice', age: 31 }
    ```

* **Spread Operator in Function Calls**

    ```js
    // Using spread operator to pass array elements as function arguments
    function add(a, b, c) {
    return a + b + c;
    }

    const numbers = [1, 2, 3];
    console.log(add(...numbers)); // 6

    // Before ES6
    // console.log(add.apply(null, numbers));
    ```

### Practical Example: Data Processing Function

```js
    // A function that processes user data with various parameter options
    function processUserData(
    { name, email, role = 'User', active = true }, // Default values in object destructuring
    ...permissions // Rest parameter for permissions
    ) {
    // Create a new user object with spread operator
    const user = {
        ...{ name, email, role, active },
        permissions,
        createdAt: new Date().toISOString()
    };

    return user;
    }

    const userData = { name: 'Alice', email: 'alice@example.com' };
    const userWithPermissions = processUserData(userData, 'read', 'write');

    console.log(userWithPermissions);
    /* Output:
    {
    name: 'Alice',
    email: 'alice@example.com',
    role: 'User',
    active: true,
    permissions: ['read', 'write'],
    createdAt: '2023-05-15T...'
    }
    */
```